In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.utils import *
from sklearn.preprocessing import *
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Bidirectional
from keras.callbacks import EarlyStopping
from keras.models import load_model

Using TensorFlow backend.
C:\Users\admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
def make_df(DataFrame) : 
    df = pd.DataFrame()

    df['Confirm'] = (DataFrame['Daily_confirmed'] + DataFrame['Daily_deaths']) / DataFrame['Total_confirmed']
    df['Flight'] = DataFrame['Passenger_arrivals'] / DataFrame['Flight_arrivals']
    df['Ship'] = ( DataFrame['Ship_crew(korea)'] + DataFrame['Ship_crew(foreign)'] + DataFrame['Ship_passenger'] ) / DataFrame['Ship_count']
    df['News'] = DataFrame['News_Negative'] / DataFrame['News_Sum']
    df['Date'] = DataFrame['Date'].astype(int)

    df.fillna(0, inplace = True)
    df.set_index('Date',inplace=True)

    df = df.loc[20200101:20200428]
    
    Flight_df = pd.DataFrame(df['Flight'])
    Flight_df.columns = ['Flight']

    Ship_df = pd.DataFrame(df['Ship'])
    Ship_df.columns = ['Ship']

    Confirm_df = pd.DataFrame(df['Confirm'])
    Confirm_df.columns = ['Confirm']

    News_df = pd.DataFrame(df['News'])
    News_df.columns = ['News']
    
    return Flight_df, Ship_df, Confirm_df, News_df

In [3]:
def make_x_train_test(DataFrame, name) : 
    
    df = DataFrame[:][:]
    
    shift_size = 7
    
    for s in range(1, shift_size+1):
        df['shift_{}'.format(s)] = df[name].shift(s)
          
    y_train = np.asarray(df[name][shift_size:-1])
    
    df.drop( name, axis=1, inplace=True)
    
    X_train = np.array(df[shift_size:-1])
    X_test = np.array(df[-1:])

    X_train_r = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
    X_test_r = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
      
    return X_train_r, X_test_r, y_train

In [4]:
def model ( X_train, X_test, y_train, save_path ) : 

    K.clear_session()
    xInput = Input(batch_shape=(None, X_train.shape[1], X_train.shape[2]))
    xLstm_1 = LSTM(20, return_sequences = True)(xInput)
    xLstm_2 = Bidirectional(LSTM(20))(xLstm_1)
    xOutput = Dense(1)(xLstm_2)

    model = Model(xInput, xOutput)
    model.compile(loss='mse', optimizer='adam',  metrics = [ 'mae' ])
    
    early_stop = EarlyStopping(monitor='loss', patience=10, mode='min', verbose=2)

    model.fit(X_train, y_train, epochs=2000, batch_size=1, verbose=2, shuffle=False, callbacks=[early_stop])

    y_hat = model.predict(X_test, batch_size=1)
    
    model.save('../model/'+save_path+'.h5')
       
    return y_hat

In [5]:
China_df = pd.read_csv('../data/Concat/China_concat.csv', header=0)
Asia_df = pd.read_csv('../data/Concat/Asia_concat.csv', header=0)
America_df = pd.read_csv('../data/Concat/America_concat.csv', header=0)
Africa_df = pd.read_csv('../data/Concat/Africa_concat.csv', header=0)
Europe_df = pd.read_csv('../data/Concat/Europe_concat.csv', header=0)
Oceania_df = pd.read_csv('../data/Concat/Oceania_concat.csv', header=0)

In [6]:
Continent_list = [ China_df, Asia_df, America_df, Africa_df, Europe_df, Oceania_df ]
Continent_name_list = [ 'China', 'Asia', 'America', 'Africa', 'Europe', 'Oceania' ] 
df_name_list = [ 'Flight', 'Ship', 'Confirm', 'News' ]

In [7]:
for Continent_index in range(len(Continent_list)) : 
    Flight_df, Ship_df, Confirm_df, News_df = make_df(Continent_list[Continent_index])
    df_list = [ Flight_df, Ship_df, Confirm_df, News_df ]
    
    for df_index in range(len(df_list)) :
        df = df_list[df_index]
        sc = MinMaxScaler()
        df = sc.fit_transform(df)
        df = pd.DataFrame(df, columns=[df_name_list[df_index]])

        date_list = [ len(df)+1, len(df)+2, len(df)+3, len(df)+4, len(df)+5, len(df)+6, len(df)+7 ]

        for date in date_list : 
            df.loc[date] = 0
            X_train, X_test, y_train = make_x_train_test(df, df_name_list[df_index])
            path = Continent_name_list[Continent_index]+df_name_list[df_index]
            y_hat = model(X_train, X_test, y_train, path)
            df.loc[date] = y_hat

        df_list[df_index]=df[:]
    
    test_df = pd.DataFrame()    
    test_df = pd.concat( [ df_list[0].tail(7), df_list[1].tail(7), df_list[2].tail(7), df_list[3].tail(7) ] , axis=1)
    test_df.to_csv('../data/Test/'+Continent_name_list[Continent_index]+ '_test_data.csv', index=False)
        


Epoch 1/2000
 - 1s - loss: 0.1015 - mae: 0.1959
Epoch 2/2000
 - 0s - loss: 0.0335 - mae: 0.1300
Epoch 3/2000
 - 0s - loss: 0.0249 - mae: 0.1160
Epoch 4/2000
 - 0s - loss: 0.0200 - mae: 0.1066
Epoch 5/2000
 - 0s - loss: 0.0185 - mae: 0.1036
Epoch 6/2000
 - 0s - loss: 0.0176 - mae: 0.1019
Epoch 7/2000
 - 0s - loss: 0.0176 - mae: 0.1023
Epoch 8/2000
 - 0s - loss: 0.0175 - mae: 0.1023
Epoch 9/2000
 - 0s - loss: 0.0177 - mae: 0.1029
Epoch 10/2000
 - 0s - loss: 0.0175 - mae: 0.1025
Epoch 11/2000
 - 0s - loss: 0.0177 - mae: 0.1027
Epoch 12/2000
 - 0s - loss: 0.0174 - mae: 0.1017
Epoch 13/2000
 - 0s - loss: 0.0174 - mae: 0.1017
Epoch 14/2000
 - 0s - loss: 0.0170 - mae: 0.1007
Epoch 15/2000
 - 0s - loss: 0.0168 - mae: 0.1000
Epoch 16/2000
 - 0s - loss: 0.0167 - mae: 0.0995
Epoch 17/2000
 - 0s - loss: 0.0161 - mae: 0.0979
Epoch 18/2000
 - 0s - loss: 0.0158 - mae: 0.0973
Epoch 19/2000
 - 0s - loss: 0.0151 - mae: 0.0954
Epoch 20/2000
 - 0s - loss: 0.0148 - mae: 0.0945
Epoch 21/2000
 - 0s - loss: 

Epoch 163/2000
 - 0s - loss: 0.0030 - mae: 0.0444
Epoch 164/2000
 - 0s - loss: 0.0030 - mae: 0.0443
Epoch 165/2000
 - 0s - loss: 0.0030 - mae: 0.0442
Epoch 166/2000
 - 0s - loss: 0.0030 - mae: 0.0440
Epoch 167/2000
 - 0s - loss: 0.0029 - mae: 0.0439
Epoch 168/2000
 - 0s - loss: 0.0029 - mae: 0.0437
Epoch 169/2000
 - 0s - loss: 0.0029 - mae: 0.0436
Epoch 170/2000
 - 0s - loss: 0.0029 - mae: 0.0434
Epoch 171/2000
 - 0s - loss: 0.0029 - mae: 0.0432
Epoch 172/2000
 - 0s - loss: 0.0028 - mae: 0.0430
Epoch 173/2000
 - 0s - loss: 0.0028 - mae: 0.0428
Epoch 174/2000
 - 0s - loss: 0.0028 - mae: 0.0426
Epoch 175/2000
 - 0s - loss: 0.0028 - mae: 0.0425
Epoch 176/2000
 - 0s - loss: 0.0028 - mae: 0.0423
Epoch 177/2000
 - 0s - loss: 0.0028 - mae: 0.0421
Epoch 178/2000
 - 0s - loss: 0.0027 - mae: 0.0419
Epoch 179/2000
 - 0s - loss: 0.0027 - mae: 0.0417
Epoch 180/2000
 - 0s - loss: 0.0027 - mae: 0.0415
Epoch 181/2000
 - 0s - loss: 0.0027 - mae: 0.0413
Epoch 182/2000
 - 0s - loss: 0.0027 - mae: 0.0411


Epoch 28/2000
 - 0s - loss: 0.0110 - mae: 0.0840
Epoch 29/2000
 - 0s - loss: 0.0106 - mae: 0.0828
Epoch 30/2000
 - 0s - loss: 0.0103 - mae: 0.0817
Epoch 31/2000
 - 0s - loss: 0.0100 - mae: 0.0806
Epoch 32/2000
 - 0s - loss: 0.0097 - mae: 0.0796
Epoch 33/2000
 - 0s - loss: 0.0094 - mae: 0.0786
Epoch 34/2000
 - 0s - loss: 0.0091 - mae: 0.0777
Epoch 35/2000
 - 0s - loss: 0.0089 - mae: 0.0768
Epoch 36/2000
 - 0s - loss: 0.0087 - mae: 0.0759
Epoch 37/2000
 - 0s - loss: 0.0085 - mae: 0.0750
Epoch 38/2000
 - 0s - loss: 0.0083 - mae: 0.0742
Epoch 39/2000
 - 0s - loss: 0.0081 - mae: 0.0735
Epoch 40/2000
 - 0s - loss: 0.0080 - mae: 0.0727
Epoch 41/2000
 - 0s - loss: 0.0078 - mae: 0.0720
Epoch 42/2000
 - 0s - loss: 0.0076 - mae: 0.0713
Epoch 43/2000
 - 0s - loss: 0.0075 - mae: 0.0706
Epoch 44/2000
 - 0s - loss: 0.0073 - mae: 0.0700
Epoch 45/2000
 - 0s - loss: 0.0072 - mae: 0.0693
Epoch 46/2000
 - 0s - loss: 0.0071 - mae: 0.0687
Epoch 47/2000
 - 0s - loss: 0.0069 - mae: 0.0680
Epoch 48/2000
 - 0s 

 - 0s - loss: 0.0026 - mae: 0.0404
Epoch 194/2000
 - 0s - loss: 0.0026 - mae: 0.0404
Epoch 195/2000
 - 0s - loss: 0.0026 - mae: 0.0403
Epoch 196/2000
 - 0s - loss: 0.0026 - mae: 0.0402
Epoch 197/2000
 - 0s - loss: 0.0026 - mae: 0.0402
Epoch 198/2000
 - 0s - loss: 0.0026 - mae: 0.0401
Epoch 199/2000
 - 0s - loss: 0.0025 - mae: 0.0400
Epoch 200/2000
 - 0s - loss: 0.0025 - mae: 0.0399
Epoch 201/2000
 - 0s - loss: 0.0025 - mae: 0.0399
Epoch 202/2000
 - 0s - loss: 0.0025 - mae: 0.0398
Epoch 203/2000
 - 0s - loss: 0.0025 - mae: 0.0397
Epoch 204/2000
 - 0s - loss: 0.0025 - mae: 0.0396
Epoch 205/2000
 - 0s - loss: 0.0025 - mae: 0.0396
Epoch 206/2000
 - 0s - loss: 0.0025 - mae: 0.0395
Epoch 207/2000
 - 0s - loss: 0.0025 - mae: 0.0394
Epoch 208/2000
 - 0s - loss: 0.0025 - mae: 0.0394
Epoch 209/2000
 - 0s - loss: 0.0025 - mae: 0.0393
Epoch 210/2000
 - 0s - loss: 0.0025 - mae: 0.0392
Epoch 211/2000
 - 0s - loss: 0.0025 - mae: 0.0391
Epoch 212/2000
 - 0s - loss: 0.0024 - mae: 0.0391
Epoch 213/2000


Epoch 79/2000
 - 0s - loss: 0.0043 - mae: 0.0519
Epoch 80/2000
 - 0s - loss: 0.0042 - mae: 0.0518
Epoch 81/2000
 - 0s - loss: 0.0042 - mae: 0.0516
Epoch 82/2000
 - 0s - loss: 0.0042 - mae: 0.0514
Epoch 83/2000
 - 0s - loss: 0.0041 - mae: 0.0512
Epoch 84/2000
 - 0s - loss: 0.0041 - mae: 0.0511
Epoch 85/2000
 - 0s - loss: 0.0041 - mae: 0.0509
Epoch 86/2000
 - 0s - loss: 0.0041 - mae: 0.0508
Epoch 87/2000
 - 0s - loss: 0.0040 - mae: 0.0506
Epoch 88/2000
 - 0s - loss: 0.0040 - mae: 0.0505
Epoch 89/2000
 - 0s - loss: 0.0040 - mae: 0.0504
Epoch 90/2000
 - 0s - loss: 0.0040 - mae: 0.0503
Epoch 91/2000
 - 0s - loss: 0.0040 - mae: 0.0501
Epoch 92/2000
 - 0s - loss: 0.0039 - mae: 0.0500
Epoch 93/2000
 - 0s - loss: 0.0039 - mae: 0.0499
Epoch 94/2000
 - 0s - loss: 0.0039 - mae: 0.0498
Epoch 95/2000
 - 0s - loss: 0.0039 - mae: 0.0497
Epoch 96/2000
 - 0s - loss: 0.0039 - mae: 0.0495
Epoch 97/2000
 - 0s - loss: 0.0039 - mae: 0.0494
Epoch 98/2000
 - 0s - loss: 0.0038 - mae: 0.0493
Epoch 99/2000
 - 0s 

...

Epoch 154/2000
 - 0s - loss: 0.0174 - mae: 0.0616
Epoch 155/2000
 - 0s - loss: 0.0183 - mae: 0.0656
Epoch 156/2000
 - 0s - loss: 0.0179 - mae: 0.0631
Epoch 157/2000
 - 0s - loss: 0.0165 - mae: 0.0629
Epoch 158/2000
 - 0s - loss: 0.0173 - mae: 0.0637
Epoch 159/2000
 - 0s - loss: 0.0185 - mae: 0.0644
Epoch 160/2000
 - 0s - loss: 0.0153 - mae: 0.0594
Epoch 161/2000
 - 0s - loss: 0.0163 - mae: 0.0595
Epoch 162/2000
 - 0s - loss: 0.0211 - mae: 0.0624
Epoch 163/2000
 - 0s - loss: 0.0202 - mae: 0.0610
Epoch 164/2000
 - 0s - loss: 0.0186 - mae: 0.0613
Epoch 165/2000
 - 0s - loss: 0.0159 - mae: 0.0631
Epoch 166/2000
 - 0s - loss: 0.0149 - mae: 0.0598
Epoch 167/2000
 - 0s - loss: 0.0155 - mae: 0.0609
Epoch 168/2000
 - 0s - loss: 0.0152 - mae: 0.0612
Epoch 169/2000
 - 0s - loss: 0.0143 - mae: 0.0584
Epoch 170/2000
 - 0s - loss: 0.0149 - mae: 0.0572
Epoch 171/2000
 - 0s - loss: 0.0195 - mae: 0.0648
Epoch 172/2000
 - 0s - loss: 0.0153 - mae: 0.0616
Epoch 173/2000
 - 0s - loss: 0.0141 - mae: 0.0588


Epoch 122/2000
 - 0s - loss: 0.0203 - mae: 0.0638
Epoch 123/2000
 - 0s - loss: 0.0204 - mae: 0.0647
Epoch 124/2000
 - 0s - loss: 0.0200 - mae: 0.0633
Epoch 125/2000
 - 0s - loss: 0.0202 - mae: 0.0635
Epoch 126/2000
 - 0s - loss: 0.0201 - mae: 0.0635
Epoch 127/2000
 - 0s - loss: 0.0202 - mae: 0.0669
Epoch 128/2000
 - 0s - loss: 0.0204 - mae: 0.0641
Epoch 129/2000
 - 0s - loss: 0.0198 - mae: 0.0639
Epoch 130/2000
 - 0s - loss: 0.0201 - mae: 0.0643
Epoch 131/2000
 - 0s - loss: 0.0197 - mae: 0.0653
Epoch 132/2000
 - 0s - loss: 0.0212 - mae: 0.0693
Epoch 133/2000
 - 0s - loss: 0.0206 - mae: 0.0692
Epoch 134/2000
 - 0s - loss: 0.0203 - mae: 0.0655
Epoch 135/2000
 - 0s - loss: 0.0196 - mae: 0.0650
Epoch 136/2000
 - 0s - loss: 0.0198 - mae: 0.0640
Epoch 137/2000
 - 0s - loss: 0.0192 - mae: 0.0620
Epoch 138/2000
 - 0s - loss: 0.0195 - mae: 0.0625
Epoch 139/2000
 - 0s - loss: 0.0191 - mae: 0.0621
Epoch 140/2000
 - 0s - loss: 0.0192 - mae: 0.0619
Epoch 141/2000
 - 0s - loss: 0.0190 - mae: 0.0622


 - 0s - loss: 0.0212 - mae: 0.0681
Epoch 83/2000
 - 0s - loss: 0.0211 - mae: 0.0679
Epoch 84/2000
 - 0s - loss: 0.0211 - mae: 0.0678
Epoch 85/2000
 - 0s - loss: 0.0210 - mae: 0.0674
Epoch 86/2000
 - 0s - loss: 0.0213 - mae: 0.0690
Epoch 87/2000
 - 0s - loss: 0.0215 - mae: 0.0712
Epoch 88/2000
 - 0s - loss: 0.0209 - mae: 0.0684
Epoch 89/2000
 - 0s - loss: 0.0210 - mae: 0.0671
Epoch 90/2000
 - 0s - loss: 0.0208 - mae: 0.0667
Epoch 91/2000
 - 0s - loss: 0.0211 - mae: 0.0675
Epoch 92/2000
 - 0s - loss: 0.0210 - mae: 0.0685
Epoch 93/2000
 - 0s - loss: 0.0210 - mae: 0.0671
Epoch 94/2000
 - 0s - loss: 0.0207 - mae: 0.0666
Epoch 95/2000
 - 0s - loss: 0.0210 - mae: 0.0665
Epoch 96/2000
 - 0s - loss: 0.0207 - mae: 0.0669
Epoch 97/2000
 - 0s - loss: 0.0209 - mae: 0.0661
Epoch 98/2000
 - 0s - loss: 0.0205 - mae: 0.0656
Epoch 99/2000
 - 0s - loss: 0.0205 - mae: 0.0643
Epoch 100/2000
 - 0s - loss: 0.0214 - mae: 0.0684
Epoch 101/2000
 - 0s - loss: 0.0210 - mae: 0.0697
Epoch 102/2000
 - 0s - loss: 0.0

Epoch 69/2000
 - 0s - loss: 0.0211 - mae: 0.0695
Epoch 70/2000
 - 0s - loss: 0.0210 - mae: 0.0688
Epoch 71/2000
 - 0s - loss: 0.0209 - mae: 0.0677
Epoch 72/2000
 - 0s - loss: 0.0209 - mae: 0.0669
Epoch 73/2000
 - 0s - loss: 0.0209 - mae: 0.0670
Epoch 74/2000
 - 0s - loss: 0.0209 - mae: 0.0670
Epoch 75/2000
 - 0s - loss: 0.0208 - mae: 0.0677
Epoch 76/2000
 - 0s - loss: 0.0213 - mae: 0.0696
Epoch 77/2000
 - 0s - loss: 0.0213 - mae: 0.0705
Epoch 78/2000
 - 0s - loss: 0.0209 - mae: 0.0694
Epoch 79/2000
 - 0s - loss: 0.0207 - mae: 0.0681
Epoch 80/2000
 - 0s - loss: 0.0206 - mae: 0.0667
Epoch 81/2000
 - 0s - loss: 0.0206 - mae: 0.0654
Epoch 82/2000
 - 0s - loss: 0.0206 - mae: 0.0652
Epoch 83/2000
 - 0s - loss: 0.0205 - mae: 0.0651
Epoch 84/2000
 - 0s - loss: 0.0205 - mae: 0.0653
Epoch 85/2000
 - 0s - loss: 0.0205 - mae: 0.0654
Epoch 86/2000
 - 0s - loss: 0.0204 - mae: 0.0653
Epoch 87/2000
 - 0s - loss: 0.0206 - mae: 0.0660
Epoch 88/2000
 - 0s - loss: 0.0202 - mae: 0.0653
Epoch 89/2000
 - 0s 

Epoch 23/2000
 - 0s - loss: 0.0133 - mae: 0.0676
Epoch 24/2000
 - 0s - loss: 0.0133 - mae: 0.0676
Epoch 25/2000
 - 0s - loss: 0.0133 - mae: 0.0677
Epoch 26/2000
 - 0s - loss: 0.0133 - mae: 0.0677
Epoch 27/2000
 - 0s - loss: 0.0133 - mae: 0.0677
Epoch 28/2000
 - 0s - loss: 0.0132 - mae: 0.0677
Epoch 29/2000
 - 0s - loss: 0.0132 - mae: 0.0677
Epoch 30/2000
 - 0s - loss: 0.0132 - mae: 0.0678
Epoch 31/2000
 - 0s - loss: 0.0132 - mae: 0.0679
Epoch 32/2000
 - 0s - loss: 0.0132 - mae: 0.0679
Epoch 33/2000
 - 0s - loss: 0.0132 - mae: 0.0680
Epoch 34/2000
 - 0s - loss: 0.0132 - mae: 0.0680
Epoch 35/2000
 - 0s - loss: 0.0132 - mae: 0.0681
Epoch 36/2000
 - 0s - loss: 0.0132 - mae: 0.0681
Epoch 37/2000
 - 0s - loss: 0.0132 - mae: 0.0681
Epoch 38/2000
 - 0s - loss: 0.0132 - mae: 0.0681
Epoch 39/2000
 - 0s - loss: 0.0132 - mae: 0.0681
Epoch 40/2000
 - 0s - loss: 0.0132 - mae: 0.0681
Epoch 41/2000
 - 0s - loss: 0.0132 - mae: 0.0681
Epoch 42/2000
 - 0s - loss: 0.0132 - mae: 0.0681
Epoch 43/2000
 - 0s 